<center>
    <span style="color: #2E86C1; font-size: 3em;">
        &#128202; Preprocessing Data
    </span>
</center>

# Nội dung Notebook

Notebook này tập trung cung vào việc xử lý dữ liệu đã được EDA để trích xuất ra bộ dữ liệu ổn định về phương diện chất lượng và số lượng cho việc huấn luyện mô hình tiếp theo

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import re


In [2]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from kneed import KneeLocator

# Tổng quan notebook

| Tên đặc trưng         | Nội dung tiền xử lý                                                  |
|-----------------------|----------------------------------------------------------------------|
| **id**                | Loại bỏ                                                               |
| **sale_date**         | Chuyển đổi giá trị datetime thành ordinal                            |
| **sale_price**        | Giữ không đổi                                                              |
| **sale_nbr**          | Điền giá trị khuyết bằng median của train                            |
| **sale_warning**      | Loại bỏ                                               |
| **join_status**       | Gom nhóm đặc trưng                                          |
| **join_year**         | Loại bỏ                       |
| **latitude**          |                                                        |
| **longitude**         |                                                       |
| **area**              |                                                            |
| **city**              |                                                           |
| **zoning**            |                                     |
| **subdivision**       |                                                       |
| **present_use**       |                                           |
| **land_val**          |                                                |
| **imp_val**           |                                     |
| **year_built**        |                                                     |
| **year_reno**         |                                                         |
| **sqft_lot**          |                                                      |
| **sqft**              |                                  |
| **sqft_1**            |                                      |
| **sqft_fbsmt**        |                                                    |
| **grade**             |                                            |
| **fbsmt_grade**       |                                            |
| **condition**         |                     |
| **stories**           |                                                              |
| **beds**              |                                                             |
| **bath_full**         |                                                      |
| **bath_3qtr**         |                                                      |
| **bath_half**         |                                                      |
| **garb_sqft**         |                                                  |
| **gara_sqft**         |                                                        |
| **wfnt**              |                                                    |
| **golf**              |                                                   |
| **greenbelt**         |                    |
| **noise_traffic**     |            |
| **view_rainier**      |                                            |
| **view_olympics**     |                                        |
| **view_cascades**     |                                       |
| **view_territorial**  |                |
| **view_skyline**      |                          |
| **view_sound**        |                                 |
| **view_lakewash**     |                                         |
| **view_lakesamm**     |                                          |
| **view_otherwater**   |                                   |
| **view_other**        |               |
| **submarket**         |                                 |





| New features         | Mô tả                                                  |
|-----------------------|----------------------------------------------------------------------|
| **new_features_1**      | Đặc trưng mới được tạo từ tổng diện tích sàn và diện tích tầng hầm                |
| **new_features_2**      | Đặc trưng phân loại theo nhóm giá trị sale_price (low, medium, high)              |
| **new_features_3**      | Đặc trưng tương tác giữa số phòng ngủ và phòng tắm                                |
| **new_features_4**      | Đặc trưng xác định bất động sản có tầm nhìn đặc biệt (tổng hợp các view_*)        |
| **new_features_5**      | Đặc trưng tỷ lệ diện tích xây dựng trên diện tích đất (sqft / sqft_lot)           |

## Đọc dữ liệu

In [3]:
data_folder='../Dataset/'

In [4]:
data = pd.read_csv(data_folder + 'dataset.csv')
test_data = pd.read_csv(data_folder + 'test.csv')

In [5]:
train_data = data.drop('id', axis = 1)

# Tiền xử lý sale_date

Đưa dữ liệu ngày tháng thành giá trị số liên tục nhằm biến đổi thành đặc trưng có thể sử dụng cho huấn luyện mô hình, dữ liệu được chuyển đổi thành khoảng cách so với 1 mốc thời gian.

In [6]:
invalid__train_dates = train_data['sale_date'][~train_data['sale_date'].astype(str).str.match(r'^\d{4}-\d{2}-\d{2}$')]
print("Invalid train__sale_date formats:", invalid__train_dates.unique())


Invalid train__sale_date formats: []


In [7]:
invalid_dates_test = train_data['sale_date'][~train_data['sale_date'].astype(str).str.match(r'^\d{4}-\d{2}-\d{2}$')]
print("Invalid test__sale_date formats:", invalid_dates_test.unique())

Invalid test__sale_date formats: []


In [ ]:
train_data.loc[:, 'sale_date'] = pd.to_datetime(train_data['sale_date'])
train_data.loc[:,'sale_date'] = train_data['sale_date'].apply(lambda x: x.toordinal())

In [10]:
test_data.loc[:, 'sale_date'] = pd.to_datetime(test_data['sale_date'])
test_data.loc[:,'sale_date'] = test_data['sale_date'].apply(lambda x: x.toordinal())

# Tiền xử lý sale_price: Giữ không xoá bỏ outlier

# Tiền xử lý sale_nbr

In [12]:
print(train_data['sale_nbr'].isnull().sum())

42182


In [13]:
print(test_data['sale_nbr'].isnull().sum())

42412


In [14]:
sale_nbr_meadian=train_data['sale_nbr'].median()
train_data.loc[:, 'sale_nbr'] = train_data['sale_nbr'].fillna(sale_nbr_meadian)
test_data.loc[:, 'sale_nbr'] = test_data['sale_nbr'].fillna(sale_nbr_meadian)

# Tiền xử lý sale_warning

90% đặc trưng này chung một giá trị, 10% còn lại được phân bố cho hơn 140 giá trị categorical khác nhau đồng thời ít tương quan với thuộc tính nhãn nên xoá đi

In [ ]:
train_data=train_data.drop(columns=['sale_warning'])
test_data=test_data.drop(columns=['sale_warning'])

# Tiền xử lý join_status

In [19]:
train_data['join_status'].value_counts()

join_status
nochg               126281
new                  53085
reno - before         5864
rebuilt - before      3706
rebuilt - after       3095
demo                  2869
reno - after          2632
miss99                2468
Name: count, dtype: int64

In [20]:
test_data['join_status'].value_counts()

join_status
nochg               126519
new                  52648
reno - before         6001
rebuilt - before      3568
rebuilt - after       3146
demo                  2866
reno - after          2654
miss99                2598
Name: count, dtype: int64

In [21]:
train_data['join_status'] = train_data['join_status'].replace(
    ['miss99', 'demo', 'rebuilt - after', 'reno - after'], 'nochg'
)
test_data['join_status'] = test_data['join_status'].replace(
    ['miss99', 'demo', 'rebuilt - after', 'reno - after'], 'nochg'
)

# Tiền xử lý join_year

Đặc trưng này thể hiện tương quan kém với nhãn mục tiêu, đồng thời nội dung có thể suy ra từ đặc trưng join_status

In [26]:
train_data=train_data.drop(columns=['join_year'])
test_data=test_data.drop(columns=['join_year'])